In [1]:
import pandas as pd
import numpy as np

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from feature_engineering import functions
from utils import utils

In [4]:
# Import Christies dataset

df = pd.read_csv('./data/data-1589349272172-non-french-christies.csv')

In [5]:
# assign column names

title_desc_col = 'title'
medium_col = 'details'
dimension_col = 'details'
url_col = 'image_url'


In [7]:
df_transformed_1 = functions.text_based_transformer(df
                                        , medium=medium_col
                                        , dimensions=dimension_col
                                        , title=title_desc_col)

df_transformed_2 = functions.is_drawing(df_transformed_1)

df_transformed_2['long_list_of_things'] = df_transformed_2['details'].apply(lambda x: functions.long_list_of_things(str(x)))

df = df_transformed_2

/Users/katielazell-fairman/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/computation/expressions.py:190: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  warnings.warn(


In [8]:
# Load model
model_filename = "model_3_DecisionTreeClassifier_v2features.pkl"

with open('./models/'+ model_filename , 'rb') as file:
    model_file = pickle.load(file)

In [9]:
# Check model file
model_file

{'model': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=10, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=5,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=None, splitter='best'),
 'features': ['long_list_of_things',
  'installation_keyword',
  'print_keyword',
  'drawing_paper_word',
  'drawing_keyword',
  'painting_keyword',
  'photo_keyword',
  'has_3dimensions',
  'furniture_keyword',
  'sculpture_keyword',
  'decorative_obj_keyword',
  'decorative_obj_keyword_med',
  'is_drawing'],
 'labels': ['Drawing',
  'Installation',
  'Painting',
  'Photograph',
  'Print',
  'Sculpture',
  'decorative_object',
  'furniture']}

In [10]:
model = model_file['model']
features = model_file['features']
labels = model_file['labels']

# Check all Model Features Exist

df[features]

,long_list_of_things,installation_keyword,print_keyword,drawing_paper_word,drawing_keyword,painting_keyword,photo_keyword,has_3dimensions,furniture_keyword,sculpture_keyword,decorative_obj_keyword,decorative_obj_keyword_med,is_drawing
0,False,False,False,False,False,True,False,False,False,False,False,False,False
1,True,False,False,True,False,False,True,False,False,False,False,False,True
2,False,False,False,False,False,False,False,True,False,False,False,False,False
3,False,True,False,False,False,False,False,True,False,True,False,False,False
4,False,False,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38926,True,True,False,False,True,True,True,False,False,False,False,False,True
38927,True,False,False,False,False,False,False,True,False,True,False,False,False
38928,False,False,False,True,False,True,False,False,False,False,False,False,True
38929,True,False,False,False,True,False,True,True,False,False,False,False,True


In [13]:
# Get Test Results

results = model.predict(df[features])
y_pred = pd.DataFrame(columns=labels
                      , data=results
                      , index=df.index)

# create label for the not classified result

y_pred['no_classification'] = y_pred.apply(lambda x: 1 if sum(x) == 0 else 0, axis=1)
x = y_pred.stack()
results = pd.Series(pd.Categorical(x[x!=0].index.get_level_values(1)))

df_classifications=pd.DataFrame(results, columns=['Classification'])

In [24]:
df_classified = df.join(df_classifications)

In [25]:
df_classified

,index,sale_date,price,estimate_min,estimate_max,currency,artist,title,size,details,...,drawing_keyword,painting_keyword,photo_keyword,sculpture_keyword,decorative_obj_keyword_med,has_3dimensions,furniture_keyword,decorative_obj_keyword,is_drawing,Classification
0,24173,2010-09-22 00:00:00,16250,15000.0,20000.0,USD,Eberhard Havekost,Untitled \r,13 3/8 x 17 3/8 in. (33.9 x 45.4 cm.),"[""signed and dated Havekost DD 97 (on the reve...",...,False,True,False,False,False,False,False,False,False,Painting
1,24113,2010-09-22 00:00:00,13750,8000.0,12000.0,USD,Vito Acconci,Changing Title\r,30 x 14\u00bd in. (76.2 x 36.8 cm.),"[""signed, titled and dated Changing Title 1969...",...,False,False,True,False,False,False,False,False,True,Drawing
2,24125,2010-09-22 00:00:00,11875,12000.0,18000.0,USD,Lucio Fontana,Untitled \r,5\u00bc x 5 x 4 in. (13.3 x 12.7 x 10.1cm.),"[""incised with initials and dated LF 47 (on th...",...,False,False,False,False,False,True,False,False,False,Sculpture
3,24126,2010-09-22 00:00:00,47500,25000.0,35000.0,USD,Lynda Benglis,Untitled (Shady Grove) \r,34\u00bd x 19\u00bd x 3\u00be in. (87.6 x 49.5...,"[""signed with initials and numbered LB 282 (on...",...,False,False,False,True,False,True,False,False,False,Sculpture
4,24235,2010-11-03 00:00:00,626500,500000.0,700000.0,USD,Marc Chagall,Fleurs dans un vase u carreaux \r,19 5/8 x 16 in. (49.8 x 40.6 cm.),"[""signed Marc Chagall (lower right) \r"",""oil o...",...,False,True,False,False,False,False,False,True,False,decorative_object
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38926,24105,2010-09-22 00:00:00,80500,40000.0,60000.0,USD,Christo,Wrapped Walkways (Project for St. Stephens Gre...,28 x 22 in. (71.1 x 55.8 cm.),"[""signed and dated CHRISTO 1977 (upper right o...",...,True,True,True,False,False,False,False,False,True,Sculpture
38927,24107,2010-09-22 00:00:00,11250,10000.0,15000.0,USD,Kenny Scharf,Phorshine \r,20 x 14 x 14 in. (50.8 x 35.5 x 35.5 cm.),"[""signed, numbered and dated Kenny Scharf 96 3...",...,False,False,False,True,False,True,False,False,False,Sculpture
38928,24132,2010-09-22 00:00:00,74500,30000.0,40000.0,USD,Alexander Calder,Untitled \r,29\u00bd x 43 in. (74.9 x 109.2 cm.),"[""signed and dated Calder 66 (lower right) \r""...",...,False,True,False,False,False,False,False,False,True,Drawing
38929,24109,2010-09-22 00:00:00,25000,10000.0,15000.0,USD,Dario Robleto,"If We Fly Away, Theyll Fly Away... \r",24 x 16 x 16 in. (60.9 x 40.6 x 40. 6 cm.),"[""Bone dust, fabric, metal, silver, lead, viny...",...,True,False,True,False,False,True,False,False,True,no_classification


In [26]:
df_cat = df_classified[['index', 'Classification']].groupby('Classification').count().sort_values('index', ascending=False)


In [27]:
df_cat

,index
Classification,
Painting,17335
Drawing,9590
Sculpture,5725
no_classification,3623
Photograph,798
decorative_object,766
Installation,626
Print,328
furniture,140


In [33]:
## Check out Furniture Examples

clas = 'furniture'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=30
                     , random_state=8)

21782


23196


32203


5117


1872


...

19367


12637


20081


34739


1336


In [ ]:
# Some furniture not correct since it depects drawings of furniture

In [34]:
clas = 'Installation'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=30
                     , random_state=0)

22590


17293


28484


18220


28768


...

31014


11723


22059


2295


13781


In [ ]:
# Not great for Installation

In [38]:
clas = 'Photograph'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=30
                     , random_state=888)

38888


38124


13918


2365


472


...

23437


12228


8767


18291


14553


In [ ]:
# Pretty good for Photograph

In [40]:
clas = 'decorative_object'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=50
                     , random_state=888)

25425


14095


188


30219


5303


...

24554


38787


20495


34393


37420


In [ ]:
# Some false positives due to subject matter of painint/drawing

In [42]:
clas = 'Sculpture'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=50
                     , random_state=888)

2523


6277


32175


21157


3276


...

5851


32466


25526


29623


21928


In [ ]:
# Pretty good

In [43]:
clas = 'no_classification'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=50
                     , random_state=888)

16923


34731


31313


17690


27334


...

24912


31516


9421


38716


14822


In [ ]:
# Mixed bag

In [44]:
clas = 'Drawing'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=50
                     , random_state=888)

2334


31051


16187


14694


7503


...

14970


30896


28529


34734


8995


In [ ]:
# Not that great

In [45]:
clas = 'Painting'
utils.view_images_from_url(df_classified[df_classified['Classification'] == clas]
                     , url_col
                     , subsample=True
                     , sample_size=50
                     , random_state=888)

22578


2443


23917


33300


19886


...

35388


34758


28787


26383


29397


In [47]:
df_classified[df_classified['print_keyword']== True].shape

(546, 30)